<a href="https://colab.research.google.com/github/JooyoungO/machineLearning/blob/main/04_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **k - 최근접 이웃 분류기로 생선의 확률을 계산해보자**

**분류문제에서 클래스의 확률을 예측**

k- 최근접 이웃 분류

입력한 샘플에서 가장 가까운 k개를

5개의 특성을 기반으로 7개의 클래스의 확률을 예측해보자

https://bit.ly/hg-04-1

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


어떤 종류의 클래스들이 있는지 Species열에서 고유한 값을 추출

In [2]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


# **타깃 만들기**

Species열을 타깃으로 지정

나머지 5개 열은 입력 데이터로 지정

주의  : species 열을 선택할 때 두개의 활포를 사용하지 말 것

Species 열을 빼고 나머지 5개의 열을 선택하여 to_numpy() 메서드로 넘파이 배열로 바꾸어 fish_input 에 저장

In [3]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()

In [4]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


타깃 데이터 만들기

In [5]:
fish_target = fish['Species'].to_numpy()

훈련 세트와 테스트 세트로 나누기

In [6]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42)

In [7]:
# 정규화 & 훈련
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [8]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


분류 문제에서 score 메서드가 반환하는 값은 정확도이며 회귀 문제에서는 결정계수를 반환함

훈련 세트와 테스트 세트 점수가 낮지만 이 장의 목표는 클래스 확률을 배우는 것이므로 무시

In [10]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


# **다중분류**

타깃 데이터에 3개 이상의 클래스가 포함된 문제

이진 분류 문제에서 양성 클래스와 음성 클래스를 각각 1과 0으로 지정하여 타깃 데이터를 생성

다중 분류 문제에서도 타깃값을 숫자로 바꾸어 입력할 수도 있음

그러나 사이킷 런에서는 문자열로 된 타깂값을 그대로 사용가능

In [11]:
import numpy as np

proba = kn.predict_proba(test_scaled[:5]) # 클래스별 확률값을 반환
print(np.round(proba, decimals=4))  # 소수점 네 번 째 자리까지 표기, 다섯 번째 자리에서 반올림

[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


KNeightvorsClassifier에서 정렬된 타깃값은 classes_에 저장

In [9]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [12]:
# 5번째 샘플의 최근접 이웃의 클래스 확인
distances, indexes = kn.kneighbors(test_scaled[4:5])
print(train_target[indexes])

[['Roach' 'Perch' 'Perch']]


In [13]:
distances, indexes = kn.kneighbors(test_scaled[[4]])
print(train_target[indexes])

[['Perch' 'Perch' 'Roach']]


3개의 최근접 이웃을 사용하기 때문에 가능한 확률은 0, 1/3, 2/3, 1이 전부

확률의 다양성이 부족

그렇다면 n_neighbors를 3이 아니라 더 큰 값으로 바꾸면?

확률의 다양성이 약간 증가

훈련 세트와 테스트 세트의 점수가 높은 최대 n_neighbors 값을 찾아보자

# **로지스틱 회귀**

이름은 회귀지만 실제로는 분류 문제

선형 회귀와 동일하게 선형 방정식을 학습

문제는 z가 어떤 값도 가능함

확률이 되려면 0부터 1 사이의 값이 되어야 함

z가 아주 큰 음수 일 때는 0으로 아주 큰 양수일 때는 1로 변환해주는 방법이 필요

시그모이드 함수를 사용하여 0과 1사이의 값으로

In [ ]:
# 시그모이드 함수 출력하기
import numpy as np
import matplotlib.pyplot as plt

z = np.arange(-5, 5, 0.1)
phi = 1 / (1 + np.exp(-z))

plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

도미와 빙어 데이터만 이용해서 로지스틱 회귀 이진 분류를 실습

7개의 클래스가 있는 대티ㅓ 중에서 2개의 클래스 데이터만 선택해야 함

**불리언 인덱싱**

true, false, 값을 전달하여 행을 선택하는 방법

In [14]:
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])

['A' 'C']


훈련 세트에서 도미와 빙어의 행만 선택한 배열 만들기

In [17]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
print(bream_smelt_indexes[:])

[ True False  True False False False False  True False False False  True
 False False False  True  True False False  True False  True False False
 False  True False False  True False False False False  True False False
  True  True False False False False False  True False False False False
 False  True False  True False False  True False False False  True False
 False False False False False  True False  True False False False False
 False False False False False  True False  True False False  True  True
 False False False  True False False False False False  True False False
 False  True False  True False False  True  True False False False False
 False False False False  True  True False False  True False False]


In [18]:
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]
print(target_bream_smelt[:])

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream' 'Bream' 'Bream'
 'Bream' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream' 'Smelt' 'Smelt' 'Bream'
 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream'
 'Smelt' 'Smelt' 'Bream' 'Smelt' 'Bream' 'Bream']


In [16]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [19]:
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [20]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


첫번째 열이 음성클래스(0)에 대한 확률
두번째 열이 양성 클래스(1)에 대한 확률

In [21]:
print(lr.classes_)

['Bream' 'Smelt']


In [22]:
# 방정식에 사용되는 기울기(가중치, 계수) 및 절편
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


z = -0.404 X 무게 -0.576 X 길이 - 0.663 X 대각선 - 0.013 X 높이 - 0.732 X 두께 - 2.161

In [23]:
# 로지스틱 회귀 모델로 양성클래스에 대한 z 계산
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


# 양성 클래스에 대한 확률 계산

계산한 z값에 시그모이드 함수를 적용

파이썬의 사이파이(scipy)라이브러리에서 제공하는 시그모이드 함수를 사용해보자

scipy.special.expit()

0에 가까울수록 Bream, 1에 가까울수록 Smelt

In [24]:
from scipy.special import expit

print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


# **로지스틱 회귀 다중 분류 훈련**

훈련

  * 반복적인 알고리즘을 사용함

    * 기본값은 max_iter = 100 으로서 불충분하기 때문에 1000으로 상향조정함

  * 릿지 회귀 방식(L2 규제) 와 같이 계수의 제곱을 규제함

    * 그러나 매개변수는 c이고 작을수록 규제가 커짐

      * 선형회귀에서는 alpha 이고 클수록 규제 강도가 커짐
    
    * 기본값은 c = 1이지만 규제를 느슨하게 하기 위해 c = 20으로 상향조정함

LogisticRegression의 파라미타

* solver

  * 기울기(가중치, 계수) 및 절편을 구하기 위해 사용할 수치 최적화 알고리즘 선택 가능

  * 'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'

  * 기본값은 'lbfgs'


* penalty

  * None : penalty 없음

  * l2 : L2 규제(릿지 방식), 기본값

  * l1 : L1 규제(라쏘 방식)

  * elasticnet : 엘라스틱(L1과 L2 둘 다 사용)

In [25]:
# penalty 매개변수로 규제 종류 변경 가능
# 규제를 제어하는 매개변수 C (실수가능)
#lr = LogisticRegression(penalty='l1', C=20, max_iter=1000)
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)  # 7개의 생선 데이터가 모두 들어있는 train_scaled 와 train_target을 사용

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [26]:
# 테스트 세트의 처음 5개의 샘플에 대한 예측을 출력
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [27]:
# 테스트 세트의 처음 5개의 샘플에 대한 예측 확률을 출력
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))  # 소수점 네 번째 자리에서 반올림

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


5개 샘플에 대한 예측 -> 5개의 행

7개 생선에 대한 확률을 계산 -> 7개의 열

z를 7번 계산

In [28]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [32]:
print(lr.coef_, lr.intercept_)
print(lr.coef_.shape, lr.intercept_.shape)

[[-1.49001259 -1.02909653  2.59342992  7.70358183 -1.20070797]
 [ 0.19618191 -2.01072007 -3.77974344  6.50492519 -1.99481478]
 [ 3.562807    6.34355461 -8.48969786 -5.75757213  3.79306162]
 [-0.10458363  3.60320663  3.93067948 -3.61740078 -1.7506979 ]
 [-1.40060998 -6.0750259   5.25969918 -0.87219289  1.86043812]
 [-1.38528461  1.49213609  1.39225441 -5.67734711 -4.40097409]
 [ 0.6215019  -2.32405484 -0.9066217   1.71600589  3.69369499]] [-0.09204689 -0.26289902  3.25100925 -0.14740759  2.65498221 -6.78787045
  1.38423249]
(7, 5) (7,)


In [30]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


# 확률 계산

이중 분류에서는 시그모이드 함수를 이용했지만 다중 분류에서는 소프트 맥스 함수를 사용하여 z를 확률로 변환

소프트맥스(softmax) 함수

여러개의 선형 방정식의 출력값을 0-1 사이로 압축하고 전체 합이 1이 되도록 변환

지수함수를 사용하므로 정규화된 지수함수라고도 부름

In [31]:
from scipy.special import softmax

proba = softmax(decision, axis=1) # axis 0이냐 1이냐 ✨
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


# 실습

